First, I was going to generate a sudoku to be solved programmaticaly.  However, the more I read, the more I realized it took a solver to generate a puzzle (or do it by hand).  For more information see: https://www.101computing.net/sudoku-generator-algorithm/.  So instead I am going to find a single puzzle to test the solver as I build it, then after that is compete I can generate new puzzles.  I pulled my test puzzle from an example here:  https://krazydad.com/sudoku/sfiles/KD_Sudoku_EZ_8_v1.pdf.  More often than not, sudoku solvers use a back-tracking algorithm but I am interested in simulated annealing (both back-tracking and simulated annealing may be useful in a current work project).  The inspiration for the simulated annealing comes from https://xianblog.wordpress.com/2010/02/23/sudoku-via-simulated-annealing/, https://www.researchgate.net/publication/220704743_Sudoku_Using_Parallel_Simulated_Annealing, and https://www.youtube.com/watch?v=FyyVbuLZav8.

$$\begin{aligned}
\text{Test Puzzle}
\begin{array}{|ccc|ccc|ccc|}
\hline
2 & & 5 & & & 7 & & & 6 \\
4 & &   & 9 & 6 & & & 2 & \\
  & &  & & 8 & & & 4 & 5 \\
\hline
9 & 8 & & & 7 & 4 & & & \\
5 & 7 & & 8 & & 2 & & 6 & 9 \\
  & & & 6 & 3 & & & 5 & 7 \\
\hline
7 & 5 & & & 2 & & & & \\
  & 6 & & & 5 & 1 & & & 2 \\
3 & & & 4 & & & 5 & & 8 \\
\hline
\end{array}
\end{aligned}$$

Some other interesting tips & tricks may have been implemented from here:  https://towardsdatascience.com/15-tips-and-tricks-for-jupyter-notebook-that-will-ease-your-coding-experience-e469207ac95c

A GUI was developed by looking at the following pages:  http://newcoder.io/gui/part-3/, https://stackoverflow.com/questions/48269694/sudoku-9x9-gui-grid-python-tkinter-buttons, https://www.daniweb.com/programming/software-development/tutorials/520379/how-to-write-a-sudoku-gui-in-python-wxpython, https://trevorappleton.blogspot.com/2013/10/guide-to-creating-sudoku-solver-using.html, http://buklijas.info/blog/2018/10/01/making-web-apps-with-jupyter-notebook/

I ran into similar problems as this article and found it helpful:  https://www.adrian.idv.hk/2019-01-30-simanneal/, also youtu.be/E8tkpzDne7I

# Step 0: Install Packages

In [1]:
import numpy as np
import random
import math
import statistics
import pandas as pd

from math import exp
from matplotlib import pyplot


# Step 1:  Set Up Initial Puzzle

In [2]:
grid = np.array(
    [
        [2, 0, 5, 0, 0, 7, 0, 0, 6],
        [4, 0, 0, 9, 6, 0, 0, 2, 0],
        [0, 0, 0, 0, 8, 0, 0, 4, 5],
        [9, 8, 0, 0, 7, 4, 0, 0, 0],
        [5, 7, 0, 8, 0, 2, 0, 6, 9],
        [0, 0, 0, 6, 3, 0, 0, 5, 7],
        [7, 5, 0, 0, 2, 0, 0, 0, 0],
        [0, 6, 0, 0, 5, 1, 0, 0, 2],
        [3, 0, 0, 4, 0, 0, 5, 0, 8],
    ]
)
# print(grid)

testgrid = np.empty_like(grid)
testgrid[:] = grid

nonzerocount = np.count_nonzero(grid)
zerocount = 81 - nonzerocount

gridaddresses = [[-1, -1]]

for i in range(0, 9):
    # print(grid)
    for j in range(0, 9):
        # print(i,j,grid[i, j])
        # print(i,j,grid[0:i+1,j])
        # if testgrid[i][j] != 0:
        #    testgrid[i][j] = 1
        if grid[i, j] == 0:

            # print(grid[i,j])
            if j >= 0:
                # print(grid[i,j])
                checkset = set(grid[i, :])
                gridaddresses = np.vstack([gridaddresses, [i, j]])
                rndnumforgrid = random.randint(1, 9)
                grid[i, j] = rndnumforgrid
                # print(i, j)
                # print(grid[i,0:j])
                # print(i, j, checkset, grid[i, :])
                # print(grid[i][j])
                # print(grid[i][j] in checkset)
                # ******** THE SECTION BELOW ISN'T QUITE RIGHT, FIX IT!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!**************
                if len(checkset) <= 9 and 0 in checkset:
                    while grid[i, j] in checkset:
                        # print(checkset, grid[i, :], i, j)
                        # print(grid[i,:])
                        rndnumforgrid = random.randint(1, 9)
                        grid[i, j] = rndnumforgrid

                # print(i, j, checkset, grid[i, :])

# print(grid)
# print(testgrid)

# Step 2:  Calculate Cost Function

In [3]:
# Define a function to Calculate Cost Function
def CostFunction(ingrid):
    CostFuncVal = 0
    for i in range(0, 9):
        CostFuncVal = CostFuncVal + 9 - len(set(ingrid[i, :]))
        # print("row", i, CostFuncVal, len(set(ingrid[i, :])), ingrid[i, :])
    for j in range(0, 9):
        CostFuncVal = CostFuncVal + 9 - len(set(ingrid[:, j]))
        # print("column",j,CostFuncVal,len(set(ingrid[:, j])),np.transpose(ingrid[:, j]))
    for iii in range(0, 3):
        for jjj in range(0, 3):
            CostFuncVal = CostFuncVal + (
                9 - len(set(ingrid[iii : iii + 3, jjj : jjj + 3].flatten()))
            )
            # print("square",iii,jjj,CostFuncVal,len(set(ingrid[iii : iii + 3, jjj : jjj + 3].flatten())),ingrid[iii : iii + 3, jjj : jjj + 3].flatten(),)
    return CostFuncVal


CostFunction(grid)

44

In [4]:
def sim_anneal_iter(input_grid,var_grid,t):  
#input grid is the puzzle to be solved, var_grid is the unit grid showing which positions are variables
    iniCost = CostFunction(input_grid)
    
    randmethod = random.randint(0,1)
    #print(randmethod)
    
    if randmethod == 0:
        randrow = random.randint(0, 8)
        rowzeroidxs = np.nonzero(var_grid[randrow,] == 0)[0]

        idx1 = random.randint(0, len(rowzeroidxs) - 1)
        idx2 = random.randint(0, len(rowzeroidxs) - 1)

        pos1 = rowzeroidxs[idx1]
        pos2 = rowzeroidxs[idx2]

        while pos1 == pos2:
            pos2 = rowzeroidxs[random.randint(0, len(rowzeroidxs) - 1)]

        if pos1 != pos2:
            gridval1 = input_grid[randrow, pos1]
            gridval2 = input_grid[randrow, pos2]
            input_grid[randrow, pos1] = gridval2
            input_grid[randrow, pos2] = gridval1

        checkCost = CostFunction(input_grid)

        diff = checkCost - iniCost
        #print(diff,t)
        # calculate metropolis acceptance criterion
        if diff >= 0:
            metropolis = exp(-diff / t)
    
            #check if we the new point is better, otherwise switch back to initial
            metcheck = random.random()
            if metcheck >= metropolis:
                input_grid[randrow, pos1] = gridval1
                input_grid[randrow, pos2] = gridval2

            if metcheck < metropolis:
                iniCost = checkCost
                
        if diff < 0:
            iniCost = checkCost
        
    elif randmethod == 1:
        randcol = random.randint(0, 8)
        colzeroidxs = np.nonzero(var_grid[randcol,] == 0)[0]

        idx1 = random.randint(0, len(colzeroidxs) - 1)
        idx2 = random.randint(0, len(colzeroidxs) - 1)

        pos1 = colzeroidxs[idx1]
        pos2 = colzeroidxs[idx2]

        while pos1 == pos2:
            pos2 = colzeroidxs[random.randint(0, len(colzeroidxs) - 1)]

        if pos1 != pos2:
            gridval1 = input_grid[pos1, randcol]
            gridval2 = input_grid[pos2, randcol]
            input_grid[pos1, randcol] = gridval2
            input_grid[pos2, randcol] = gridval1

        checkCost = CostFunction(input_grid)

        diff = checkCost - iniCost
        #print(diff,t)
        # calculate metropolis acceptance criterion
        if diff >=0:
            metropolis = exp(-diff / t)
    
            #check if we the new point is better, otherwise switch back to initial
            metcheck = random.random()
            if metcheck >= metropolis:
                input_grid[pos1, randcol] = gridval1
                input_grid[pos2, randcol] = gridval2

            if metcheck < metropolis:
                iniCost = checkCost
        if diff < 0:
            iniCost = checkCost
        
    elif randmethod == 2:
        randsqr1 = random.randint(0, 2)
        randsqr2 = random.randint(0,2)
        # ******************MOD IS NOT WORKING RIGHT HERE, LOOK CLOSER*********************************
        sqrzeroidxs = np.argwhere(var_grid[(randsqr1)*3:(randsqr1)*3+3,(randsqr2)*3:(randsqr2)*3+3] == 0)
        #sqrnonzeroidxs = np.transpose(np.nonzero(var_grid[(randsqr1)*3:(randsqr1)*3+3,(randsqr2)*3:(randsqr2)*3+3] == 0))
        #allidxs = np.array([[0,0],[0,1],[0,2],[1,0],[1,1],[1,2],[2,0],[2,1],[2,2]])
        #filtered = filter(lambda x:x not in sqrnonzeroidxs,allidxs)
        #print(list(filtered))
        #sqrzeroidxs = set(allidxs) - set(sqrnonzeroidxs)
        #print(len(sqrzeroidxs))
        #print(var_grid)
        #print(sqrnonzeroidxs)
        print(sqrzeroidxs)
        #print((randsqr % 3)*3, (randsqr % 3)*3+3, math.floor(randsqr/3)*3, math.floor(randsqr/3)*3+3)
        print(input_grid[(randsqr1)*3:(randsqr1)*3+3,(randsqr2)*3:(randsqr2)*3+3])
        print(var_grid[(randsqr1)*3:(randsqr1)*3+3,(randsqr2)*3:(randsqr2)*3+3])
        
        idx1 = random.randint(0, len(sqrzeroidxs) - 1)
        idx2 = random.randint(0, len(sqrzeroidxs) - 1)

        pos1 = sqrzeroidxs[idx1,]
        pos2 = sqrzeroidxs[idx2,]
        
        #print(t,randsqr1,randsqr2,pos1,pos2,sqrzeroidxs)
        
        while pos1[0] == pos2[0] and pos1[1] == pos2[1]:
            pos2 = sqrzeroidxs[random.randint(0, len(sqrzeroidxs) - 1)]

        if pos1[0] != pos2[0] and pos1[1] != pos2[1]:
            gridval1 = input_grid[(randsqr1)*3 + pos1[0],(randsqr2)*3 + pos1[1]]
            gridval2 = input_grid[(randsqr1)*3 + pos2[0],(randsqr2)*3 + pos2[1]]
            input_grid[(randsqr1)*3 + pos1[0],(randsqr2)*3 + pos1[1]] = gridval2
            input_grid[(randsqr1)*3 + pos2[0],(randsqr2)*3 + pos2[1]] = gridval1

        checkCost = CostFunction(input_grid)

        diff = checkCost - iniCost
        #print(diff, t)
        #calculate metropolis acceptance criterion
        if diff >= 0:
            metropolis = exp(-diff / t)
    
            #check if we the new point is better, otherwise switch back to initial
            metcheck = random.random()
            if metcheck >= metropolis:
                input_grid[(randsqr1)*3 + pos1[0],(randsqr2)*3 + pos1[1]] = gridval1
                input_grid[(randsqr1)*3 + pos2[0],(randsqr2)*3 + pos2[1]] = gridval2

            if metcheck < metropolis:
                iniCost = checkCost
    
        if diff < 0:
            iniCost = checkCost
    #print(t,iniCost, checkCost, diff,metropolis,metcheck,t)


In [5]:
# launch window
# main
# initial temperature
temp = 0.3

# define the total iterations
n_iterations = 250000000

tt = temp

# run the algorithm
for i in range(n_iterations):
    # calculate temperature for current epoch
    tt = tt * (1-1/n_iterations)  # / (1 + math.log(i + 1) / 10)  #** (math.floor((i + 1) / 10))-/
    # print(tt)

    #    print(CostFunction(grid))
    CostCheck = CostFunction(grid)
    if i % 100000 == 0:
        print(tt, CostCheck, i)
    if CostCheck == 0:
        End
    # swap two numbers in solution
    sim_anneal_iter(grid, testgrid, tt)

# print(grid)
# for i in range(0, 9):
# print(i, set(grid[i, :]), grid[i, :])
# for j in range(0, 9):
# print(j, set(grid[:, j]), grid[:, j])
# for iii in range(0, 3):
# for jjj in range(0, 3):
# print(
#    iii,
#    jjj,
#    set(grid[iii : iii + 3, jjj : jjj + 3].flatten()),
#    grid[iii : iii + 3, jjj : jjj + 3].flatten(),
# )

CostFuncVal = 0
for i in range(0, 9):
    CostFuncVal = CostFuncVal + 9 - len(set(grid[i, :]))
    print("row", i, CostFuncVal, len(set(grid[i, :])), grid[i, :])
for j in range(0, 9):
    CostFuncVal = CostFuncVal + 9 - len(set(grid[:, j]))
    print("column", j, CostFuncVal, len(set(grid[:, j])), np.transpose(grid[:, j]))
for iii in range(0, 3):
    for jjj in range(0, 3):
        CostFuncVal = (
            CostFuncVal + 9 - len(set(grid[iii : iii + 3, jjj : jjj + 3].flatten()))
        )
        print(
            "square",
            iii,
            jjj,
            CostFuncVal,
            len(set(grid[iii : iii + 3, jjj : jjj + 3].flatten())),
            grid[iii : iii + 3, jjj : jjj + 3].flatten(),
        )

print(grid)
print(testgrid)

0.2999999988 44 0
0.2998800227971035 19 100000
0.2997600947750113 15 200000
0.29964021471453517 17 300000
0.2995203825964945 15 400000
0.299400598401716 14 500000
0.29928086211103405 18 600000
0.29916117370529033 17 700000
0.2990415331653365 18 800000
0.2989219404720274 13 900000
0.2988023956062303 14 1000000
0.29868289854881563 16 1100000
0.29856344928066647 17 1200000
0.2984440477826707 21 1300000
0.2983246940357219 18 1400000
0.2982053880207255 11 1500000
0.2980861297185919 10 1600000
0.29796691911024065 11 1700000
0.29784775617659653 12 1800000
0.2977286408985937 16 1900000
0.29760957325717413 17 2000000
0.2974905532332876 17 2100000
0.29737158080789133 15 2200000
0.29725265596194783 18 2300000
0.29713377867643 16 2400000
0.2970149489323178 15 2500000
0.2968961667105977 15 2600000
0.2967774319922653 14 2700000
0.296658744758323 13 2800000
0.2965401049897803 11 2900000
0.29642151266765554 12 3000000
0.2963029677729736 17 3100000
0.2961844702867669 16 3200000
0.29606602019007683 16 3

0.2698273933013458 14 26500000
0.26971948392717965 12 26600000
0.2696116177081321 14 26700000
0.26950379462694396 10 26800000
0.2693960146663642 11 26900000
0.2692882778091467 10 27000000
0.26918058403805495 10 27100000
0.2690729333358576 10 27200000
0.26896532568532944 15 27300000
0.2688577610692545 10 27400000
0.26875023947042215 12 27500000
0.26864276087162836 17 27600000
0.26853532525567714 10 27700000
0.2684279326053781 15 27800000
0.26832058290355004 10 27900000
0.2682132761330152 13 28000000
0.2681060122766055 12 28100000
0.2679987913171581 10 28200000
0.26789161323751753 10 28300000
0.2677844780205361 11 28400000
0.2676773856490723 12 28500000
0.26757033610599107 10 28600000
0.2674633293741645 11 28700000
0.26735636543647084 12 28800000
0.2672494442757967 11 28900000
0.2671425658750346 13 29000000
0.2670357302170835 11 29100000
0.26692893728485034 10 29200000
0.2668221870612378 10 29300000
0.2667154795291856 10 29400000
0.2666088146716103 10 29500000
0.26650219247144596 12 2960

0.2425923518725143 8 53100000
0.24249533433642342 9 53200000
0.24239835559958647 10 53300000
0.24230141564648708 10 53400000
0.24220451446161448 11 53500000
0.24210765202946505 12 53600000
0.24201082833453935 8 53700000
0.24191404336134795 8 53800000
0.24181729709440414 8 53900000
0.24172058951822847 8 54000000
0.2416239206173477 11 54100000
0.24152729037629408 8 54200000
0.24143069877960846 8 54300000
0.2413341458118355 8 54400000
0.24123763145752605 9 54500000
0.24114115570123873 8 54600000
0.24104471852753656 8 54700000
0.24094831992099006 8 54800000
0.24085195986617378 8 54900000
0.24075563834767347 8 55000000
0.24065935535007577 8 55100000
0.24056311085797555 8 55200000
0.24046690485597383 8 55300000
0.24037073732867784 8 55400000
0.2402746082606998 8 55500000
0.24017851763665962 8 55600000
0.24008246544118306 10 55700000
0.23998645165890142 8 55800000
0.2398904762744521 8 55900000
0.23979453927248062 8 56000000
0.23969864063763605 10 56100000
0.23960278035457472 8 56200000
0.2395

0.21810628071151977 8 79700000
0.2180190556452829 8 79800000
0.2179318654620956 8 79900000
0.21784471014800705 8 80000000
0.21775758968907294 8 80100000
0.21767050407135366 8 80200000
0.21758345328091563 8 80300000
0.2174964373038307 8 80400000
0.21740945612617635 8 80500000
0.21732250973403572 8 80600000
0.21723559811349707 8 80700000
0.21714872125065474 8 80800000
0.2170618791316081 8 80900000
0.21697507174246247 8 81000000
0.21688829906932858 8 81100000
0.21680156109833323 8 81200000
0.21671485781557814 8 81300000
0.21662818920720137 8 81400000
0.21654155525933555 8 81500000
0.21645495595811923 8 81600000
0.21636839128969657 8 81700000
0.2162818612402167 8 81800000
0.2161953657958351 8 81900000
0.2161089049427127 8 82000000
0.21602247866701593 10 82100000
0.21593608695491623 8 82200000
0.21584972979259098 8 82300000
0.21576340716622314 8 82400000
0.215677119062001 8 82500000
0.21559086546611825 8 82600000
0.21550464636477454 8 82700000
0.21541846174417514 8 82800000
0.21533231159052

0.19617016474147225 8 106200000
0.19609171236698114 8 106300000
0.19601329136716455 8 106400000
0.1959349017294751 9 106500000
0.1958565434413702 8 106600000
0.1957782164903129 8 106700000
0.19569992086377083 8 106800000
0.19562165654921648 8 106900000
0.19554342353412782 8 107000000
0.1954652218059871 8 107100000
0.19538705135228313 10 107200000
0.19530891216050736 8 107300000
0.19523080421815833 8 107400000
0.19515272751273874 8 107500000
0.19507468203175538 8 107600000
0.1949966677627215 8 107700000
0.19491868469315526 8 107800000
0.19484073281057912 8 107900000
0.1947628121025208 8 108000000
0.1946849225565132 8 108100000
0.19460706416009324 8 108200000
0.19452923690080434 8 108300000
0.19445144076619336 8 108400000
0.19437367574381395 9 108500000
0.19429594182122292 9 108600000
0.1942182389859828 8 108700000
0.19414056722566167 8 108800000
0.19406292652783155 8 108900000
0.19398531688007026 8 109000000
0.19390773826996038 8 109100000
0.19383019068508928 8 109200000
0.1937526741130

0.17686424658868666 8 132100000
0.17679351503720017 8 132200000
0.17672281177267646 8 132300000
0.1766521367838036 8 132400000
0.17658149005927287 8 132500000
0.1765108715877808 8 132600000
0.17644028135802867 8 132700000
0.176369719358723 8 132800000
0.17629918557857208 8 132900000
0.1762286800062914 8 133000000
0.1761582026306001 8 133100000
0.1760877534402213 8 133200000
0.17601733242388384 8 133300000
0.17594693957031982 8 133400000
0.17587657486826636 8 133500000
0.17580623830646538 8 133600000
0.17573592987366288 8 133700000
0.17566564955860917 8 133800000
0.1755953973500606 8 133900000
0.1755251732367763 8 134000000
0.17545497720752 8 134100000
0.1753848092510609 8 134200000
0.17531466935617118 8 134300000
0.1752445575116289 8 134400000
0.17517447370621606 8 134500000
0.17510441792871953 8 134600000
0.17503439016793115 8 134700000
0.17496439041264497 8 134800000
0.17489441865166194 8 134900000
0.17482447487378638 8 135000000
0.17475455906782714 8 135100000
0.17468467122259732 8 

0.1594583037773005 8 158000000
0.15939453321065944 8 158100000
0.15933078814714385 8 158200000
0.15926706857655415 8 158300000
0.1592033744886964 8 158400000
0.15913970587337858 8 158500000
0.15907606272041463 8 158600000
0.15901244501962092 8 158700000
0.1589488527608191 8 158800000
0.158885285933834 8 158900000
0.1588217445284952 8 159000000
0.15875822853463614 8 159100000
0.15869473794209402 8 159200000
0.15863127274071018 8 159300000
0.15856783292033064 8 159400000
0.15850441847080463 8 159500000
0.15844102938198587 8 159600000
0.1583776656437323 8 159700000
0.15831432724590563 8 159800000
0.15825101417837173 8 159900000
0.15818772643100043 8 160000000
0.15812446399366578 8 160100000
0.15806122685624568 8 160200000
0.15799801500862237 8 160300000
0.15793482844068185 8 160400000
0.1578716671423146 8 160500000
0.157808531103414 8 160600000
0.15774542031387956 8 160700000
0.15768233476359292 8 160800000
0.15761927444249774 8 160900000
0.15755623934048904 8 161000000
0.1574932294474784

0.14376535186708284 8 183900000
0.14370785722594595 8 184000000
0.14365038557806672 8 184100000
0.14359293691424918 8 184200000
0.14353551122530178 8 184300000
0.14347810850203702 8 184400000
0.14342072873526968 8 184500000
0.14336337191581905 8 184600000
0.14330603803450884 8 184700000
0.1432487270821649 8 184800000
0.14319143904961779 8 184900000
0.14313417392770134 8 185000000
0.14307693170725316 8 185100000
0.14301971237911493 8 185200000
0.1429625159341305 8 185300000
0.14290534236314922 8 185400000
0.14284819165702348 8 185500000
0.14279106380660808 8 185600000
0.14273395880276352 8 185700000
0.1426768766363527 8 185800000
0.14261981729824258 8 185900000
0.14256278077930284 8 186000000
0.14250576707040907 8 186100000
0.1424487761624385 8 186200000
0.1423918080462723 8 186300000
0.14233486271279605 8 186400000
0.14227794015289816 8 186500000
0.14222104035747088 8 186600000
0.1421641633174113 8 186700000
0.14210730902361737 8 186800000
0.14205047746699306 8 186900000
0.141993668638

0.12961680833085043 8 209800000
0.1295649719754039 8 209900000
0.1295131563503534 8 210000000
0.12946136144740797 8 210100000
0.1294095872582813 8 210200000
0.12935783377468854 8 210300000
0.12930610098834952 8 210400000
0.12925438889098675 8 210500000
0.1292026974743269 8 210600000
0.1291510267300985 8 210700000
0.12909937665003507 8 210800000
0.12904774722587212 8 210900000
0.1289961384493492 8 211000000
0.12894455031220878 8 211100000
0.12889298280619657 8 211200000
0.12884143592306213 8 211300000
0.12878990965455753 8 211400000
0.128738403992439 8 211500000
0.12868691892846554 8 211600000
0.1286354544543994 8 211700000
0.12858401056200633 8 211800000
0.12853258724305505 8 211900000
0.12848118448932141 8 212000000
0.12842980229257417 8 212100000
0.12837844064459583 8 212200000
0.12832709953716845 8 212300000
0.12827577896207726 8 212400000
0.12822447891111127 8 212500000
0.128173199376062 8 212600000
0.12812194034872507 8 212700000
0.1280707018208988 8 212800000
0.12801948378438532 

0.11690743516872333 8 235600000
0.11686068154593506 8 235700000
0.1168139466208559 8 235800000
0.11676723038600854 8 235900000
0.11672053283391808 8 236000000
0.11667385395711352 8 236100000
0.11662719374812576 8 236200000
0.11658055219948943 8 236300000
0.11653392930374185 8 236400000
0.11648732505342323 8 236500000
0.11644073944107683 8 236600000
0.11639417245924916 8 236700000
0.11634762410048945 8 236800000
0.11630109435734967 8 236900000
0.11625458322238529 8 237000000
0.1162080906881547 8 237100000
0.11616161674721887 8 237200000
0.11611516139214184 8 237300000
0.11606872461549117 8 237400000
0.11602230640983663 8 237500000
0.11597590676775134 8 237600000
0.11592952568181164 8 237700000
0.11588316314459625 8 237800000
0.11583681914868729 8 237900000
0.11579049368666954 8 238000000
0.11574418675113107 8 238100000
0.11569789833466293 8 238200000
0.11565162842985914 8 238300000
0.11560537702931713 8 238400000
0.11555914412563435 8 238500000
0.11551292971141493 8 238600000
0.11546673

In [ ]:
grid = np.array(
    [
        [2, 0, 5, 0, 0, 7, 0, 0, 6],
        [4, 0, 0, 9, 6, 0, 0, 2, 0],
        [0, 0, 0, 0, 8, 0, 0, 4, 5],
        [9, 8, 0, 0, 7, 4, 0, 0, 0],
        [5, 7, 0, 8, 0, 2, 0, 6, 9],
        [0, 0, 0, 6, 3, 0, 0, 5, 7],
        [7, 5, 0, 0, 2, 0, 0, 0, 0],
        [0, 6, 0, 0, 5, 1, 0, 0, 2],
        [3, 0, 0, 4, 0, 0, 5, 0, 8],
    ]
)

print(np.argwhere(grid == 0))

In [ ]:
print(math.log(2))